In [16]:
import pandas as pd
import pyodbc


# Conexión

In [17]:
# Parámetros de conexión
server = r'CELIA\SQLEXPRESS'   # nombre del servidor
database = 'DB_NBA'            # base de datos
driver = 'ODBC Driver 17 for SQL Server' 

# Cadena de conexión
conn_str = f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"

# Conexión
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Tabla Team

In [18]:
df_team = pd.read_csv("C:/Users/Celia V/Desktop/HENRY/PROYECTO FINAL/team.csv")

for _, row in df_team.iterrows():
    cursor.execute("""
        INSERT INTO Team (team_id, full_name, abbreviation, nickname, city, state, year_founded)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, row["id"], row["full_name"], row["abbreviation"], row["nickname"],
         row["city"], row["state"], row["year_founded"])

conn.commit()

# Tabla Players

In [19]:
df_player = pd.read_csv("C:/Users/Celia V/Desktop/HENRY/PROYECTO FINAL/players_info.csv")

for _, row in df_player.iterrows():
    cursor.execute("""
        INSERT INTO Players (player_id, first_name, last_name, full_name, is_active, birthdate, country, height, weight)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, int(row["player_id"]), str(row["first_name"]), str(row["last_name"]), str(row["full_name"]), int(row["is_active"]),
        row["birthdate"] if pd.notnull(row["birthdate"]) else None, str(row["country"]), float(row["height"]), float(row["weight"])
)

conn.commit()

# Tabla Inactivos

In [20]:
df_inactive = pd.read_csv("C:/Users/Celia V/Desktop/HENRY/PROYECTO FINAL/inactive_players.csv")

# Reemplazar nulos en texto por cadena vacía
#df_inactive = df_inactive.fillna("")

for _, row in df_inactive.iterrows():
    cursor.execute("""
        INSERT INTO Inactive_Players (game_id, player_id, first_name, last_name, jersey_num, team_id, team_abbreviation)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, int(row["game_id"]), int(row["player_id"]), row["first_name"], row["last_name"], row["jersey_num"],
        int(row["team_id"]), row["team_abbreviation"])

conn.commit()

# Tabla Partidos

In [22]:
df_game = pd.read_csv("C:/Users/Celia V/Desktop/HENRY/PROYECTO FINAL/game_summary.csv")

for _, row in df_game.iterrows():
    try:
        cursor.execute("""
            INSERT INTO Game_Summary (
                game_id, game_date_est, game_sequence, game_status_id, game_status_text,
                gamecode, home_team_id, visitor_team_id, season, live_period
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, int(row["game_id"]), row["game_date_est"], int(row["game_sequence"]), int(row["game_status_id"]),
            row["game_status_text"], row["gamecode"], int(row["home_team_id"]), int(row["visitor_team_id"]),
            int(row["season"]), int(row["live_period"]))
        
    except pyodbc.IntegrityError:
        # Saltar si ya existe ese game_id
        continue

conn.commit()

# Tabla Puntos

In [23]:
df_line = pd.read_csv("C:/Users/Celia V/Desktop/HENRY/PROYECTO FINAL/line_score.csv")

for _, row in df_line.iterrows():
    cursor.execute("""
        INSERT INTO Line_Score (
            game_date_est, game_sequence, game_id, team_id_home, team_abbreviation_home,
            team_wins_losses_home, pts_qtr1_home, pts_qtr2_home, pts_qtr3_home, pts_qtr4_home,
            pts_ot1_home, pts_ot2_home, pts_ot3_home, pts_ot4_home, pts_ot5_home,
            pts_ot6_home, pts_ot7_home, pts_ot8_home, pts_ot9_home, pts_ot10_home, pts_home,
            team_id_away, team_abbreviation_away, team_wins_losses_away, pts_qtr1_away, pts_qtr2_away, pts_qtr3_away, pts_qtr4_away,
            pts_ot1_away, pts_ot2_away, pts_ot3_away, pts_ot4_away, pts_ot5_away,
            pts_ot6_away, pts_ot7_away, pts_ot8_away, pts_ot9_away, pts_ot10_away, pts_away
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()

# Tabla Estadísticas de jugador

In [26]:
df_stats = pd.read_csv("C:/Users/Celia V/Desktop/HENRY/PROYECTO FINAL/player_stats.csv")

# Renombrar columnas para que coincidan con la tabla SQL
df_stats.rename(columns={
    "FG%": "FG_pct",
    "3P": "Three_P",
    "3PA": "Three_PA",
    "3P%": "Three_P_pct",
    "2P": "Two_P",
    "2PA": "Two_PA",
    "2P%": "Two_P_pct",
    "eFG%": "eFG_pct",
    "FT%": "FT_pct"
}, inplace=True)

for _, row in df_stats.iterrows():
    cursor.execute("""
        INSERT INTO Player_Stats (
            player_id, full_name,Pos, Age, Tm, G, GS, MP, FG, FGA, FG_pct,
            Three_P, Three_PA, Three_P_pct, Two_P, Two_PA, Two_P_pct,
            eFG_pct, FT, FTA, FT_pct, ORB, DRB, TRB, AST, STL, BLK,
            TOV, PF, PTS, [Year], team_id
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, int(row["player_id"]), row["full_name"] , row["Pos"], int(row["Age"]), row["Tm"], int(row["G"]), int(row["GS"]),
         float(row["MP"]), float(row["FG"]), float(row["FGA"]), float(row["FG_pct"]),
         float(row["Three_P"]), float(row["Three_PA"]), float(row["Three_P_pct"]),
         float(row["Two_P"]), float(row["Two_PA"]), float(row["Two_P_pct"]), float(row["eFG_pct"]),
         float(row["FT"]), float(row["FTA"]), float(row["FT_pct"]), float(row["ORB"]), float(row["DRB"]),
         float(row["TRB"]), float(row["AST"]), float(row["STL"]), float(row["BLK"]), float(row["TOV"]),
         float(row["PF"]), float(row["PTS"]), str(row["Year"]), int(row["team_id"])) 

conn.commit()